<a href="https://colab.research.google.com/github/Seligmann/carvana-image-masking-challenge/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

/content/drive/My Drive/MyDrive/carvana-image-masking-challenge
